Ho 3 cell-lines, per 2 mi danno tutto e mi hanno anche separato in train and validation, per il terzo mi danno solo le x
In cage_train mi danno le x che sono dove sono i picchi di produzione per ogni gene
Poi mi danno informazioni aggiuntive in DNAsi (quanto è accessibile il DNA), e marker degli histoni (Quelli che incominciano con la H..)
La cosa importante per il risultato e che i numeri sono nell ordine giusto, non importa la magnitudo di questi numeri
Per calcolare l errore usano spearman, che controlla solo in rank

Quello che alleno sul chromosoma 1 va bene anche sul 3, oppure anche se avessero lo stesso code, non sono comparabili

In [5]:
# Import libraries that are required to run your project
# You are allowed to add more libraries as you need

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import torch
import torch.nn as nn
import pyBigWig
import glob
from tqdm import tqdm

## Work Package 1.1 - Modeling Choices & Data Pre-processing

In [1]:
SIGNALS_CNN = ['DNase', 'H3K4me1', 'H3K4me3', 'H3K27ac', 'H3K36me3']   # List of signal tracks to be used
SIGNAL_CNN_WINDOW = 1e4 # Window size in base pairs
CNN_N_BINS = 100 # Number of base pairs per bin

In [6]:
half_window = int(SIGNAL_CNN_WINDOW // 2)
chromstrs = []
ranges = []


def get_signals_bins(df: pd.DataFrame, cell_type: str) -> np.ndarray:
    df["neg_strand"] = mask = df['strand'] == '-'
    df.loc[mask, ['TSS_start', 'TSS_end']] = df.loc[mask, ['TSS_end', 'TSS_start']].values
    df["center"] = ((df["TSS_start"] + df["TSS_end"]) // 2).astype(int)
    df["window_start"] = df["center"] - half_window
    df["window_end"] = df["center"] + half_window

    bins_signal_gene = np.zeros((len(df), len(SIGNALS_CNN), CNN_N_BINS))
    for i, signal in enumerate(SIGNALS_CNN):
        print("Processing signal:", signal, f"{i+1}/{len(SIGNALS_CNN)}")

        for j, (chromstr, window_start, window_end, neg_strand) in enumerate(tqdm(df[["chr", "window_start", "window_end", "neg_strand"]].itertuples(index=False), total=len(df))):
            fname = glob.glob(f"Data/bigwig/{signal}-bigwig/{cell_type}*")[0]
            bw = pyBigWig.open(fname)

            bins = bw.stats(chromstr, window_start, window_end, type="mean", nBins=CNN_N_BINS)
            if neg_strand:
                bins = bins[::-1]
            bins_signal_gene[j, i] = bins

            bw.close()
            
    return bins_signal_gene

In [ ]:
cell_types = {
    "train": ["X1", "X2"],
    "test": ["X1", "X2"],
    "val": ["X3"]
}

for train_test_val, cell_types in cell_types.items():
    print(f"Processing {train_test_val} set.")
    for cell_type in cell_types:
        print(f"Processing cell type: {cell_type}")
        df = pd.read_csv(f'./Data/CAGE-train/{cell_type}_{train_test_val}_info.tsv', sep='\t', usecols=[0,1,4,5,6])
        signal_bins = get_signals_bins(df, cell_type)
        np.save(f'Data/processed/cnn_input_{cell_type}_{train_test_val}.npy', signal_bins)

Processing train set.
Processing cell type: X1
Processing signal: DNase 1/5


 36%|███▌      | 5100/14310 [00:13<00:24, 381.63it/s]


KeyboardInterrupt: 

## Work Package 1.2 - Model Building

In [195]:
##TRAINING AND EVALUATION LOGIC GOES HERE##


## Work Package 1.3 - Prediction on Test Data (Evaluation Metric)

In [196]:
# ---------------------------INSERT CODE HERE---------------------------




# ----------------------------------------------------------------------

# Check if "pred" meets the specified constrains
assert isinstance(pred, np.ndarray), 'Prediction array must be a numpy array'
assert np.issubdtype(pred.dtype, np.number), 'Prediction array must be numeric'
assert pred.shape[0] == len(test_genes), 'Each gene should have a unique predicted expression'

NameError: name 'pred' is not defined

In [ ]:
model.eval()  # Set the model to evaluation mode
preds = []
for X_chr, y_chr in zip(train_x, train_y):
    # Convert numpy arrays to torch tensors
    X_chr = torch.tensor(X_chr.T, dtype=torch.float32).unsqueeze(0)  # (1, 2, num_genes)
    y_chr = torch.tensor(y_chr, dtype=torch.float32).unsqueeze(0)    # (1, num_genes)

    preds.append(model(X_chr))


In [ ]:
# Convert to numpy if tensor
pred_values = preds[0].detach().cpu().numpy().ravel()  # assuming same chromosome index
gene_names_chr = train_names[0]  # assuming same chromosome index

# Build DataFrame
df_pred = pd.DataFrame({
    'gene_name': gene_names_chr,
    'prediction': pred_values
})

print(df_pred)

     gene_name   prediction
0       PRSS56  1251079.250
1        MZT2B  2166804.250
2         POMC -7441639.000
3      TMEM177  2736535.750
4        TTC7A -3640067.500
...        ...          ...
1198     CASP8 -3566437.750
1199     TRPM8  3872184.750
1200   FAM166C -8041816.000
1201   DNAJC10  1045738.125
1202   SLC20A1  2441869.500

[1203 rows x 2 columns]


#### Store Predictions in the Required Format

In [3]:
# Store predictions in a ZIP. 
# Upload this zip on the project website under "Your submission".
# Zip this notebook along with the conda environment (and README, optional) and upload this under "Your code".

save_dir = 'Data/submission'  # TODO
file_name = 'gex_predicted.csv'         # PLEASE DO NOT CHANGE THIS
zip_name = "Laffranchi_Paolo_Project1.zip" # TODO
save_path = f'{save_dir}/{zip_name}'
compression_options = dict(method="zip", archive_name=file_name)

test_genes = pd.read_csv(f"{save_dir}/{file_name}")
test_genes[['gene_name', 'gex_predicted']].to_csv(save_path, compression=compression_options)